<a href="https://colab.research.google.com/github/josephjz/Biomedical_Image_Analysis_Final/blob/main/BIAanalysis_Jennifer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Jennifer Joseph 

Date: 5/16/2022

Project: Biomedical Image Analysis- Boston College

#Uploading Data

In [6]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [7]:
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
import re
import matplotlib.pyplot as plt

#Pre-processing

In [8]:
# code from Corina
############################################################################################################
#BEFORE RUNNING: Must compile a text file with all the delays (in the format "(day)rat (ratnumber):(delay)") 
############################################################################################################
#Converts text file into an array where the days are the rows and the columns are the rats 
#The value at each position is the delay for that day (row) and rat (col)

#Creates array
reference = np.zeros((6,17))
#Accesses the text file
with open('/content/drive/MyDrive/BIA_folder/preproctimes.txt') as time_file:
    for line in time_file:
      #Finds all the numbers in the string
      nums = re.findall(r'\d+', line)

      #Obtains the dealy
      time = re.findall(r'\d+',re.findall(r'\d+-',line)[0])[0] 

      #Upates the delay in the numpy array
      reference[int(nums[0]), int(nums[1])]=int(time)

#Day 4, Rat 16 was not in the textfile so I just augmented it here
reference[4,16]='08'

In [9]:
# code from Corina
#Get video CSVs into list
framespersecond=30
file_names = [] #only getting csv files
csv_list = []
count = 0
right_left = []
for file_name in tqdm(os.listdir('/content/drive/MyDrive/BIA_folder/csvs/')): #iterating through list
  if file_name.endswith('.csv'): #only getting csv files
    file_names.append(file_name)
    csv = pd.read_csv('/content/drive/MyDrive/BIA_folder/csvs/'+file_name, dtype='a')
    numbers = re.findall(r'\d+', file_name)
    start_time = int(reference[int(numbers[1]),int(numbers[2])])
    csv.drop(csv.index[2:start_time*framespersecond],axis=0, inplace=True)
    csv.drop(csv.index[start_time*framespersecond+1:start_time*framespersecond+9001],axis=0, inplace=True)
    csv_list.append(csv.to_numpy())
    if '(1)' in file_name:
      right_left.append('R')
      count+=1
    else: 
      right_left.append('L')

100%|██████████| 129/129 [00:16<00:00,  7.63it/s]


In [10]:
# code from Corina
print(csv_list[0][0:2])
print(file_names[3]) # using this for testing freezing bc i have that video locally 

print(file_names.index('AD2205_Day2_Rat_1DLC_resnet50_BIAfinal22Apr11shuffle1_500000.csv'))

[['bodyparts' 'nose' 'nose' 'nose' 'L eye' 'L eye' 'L eye' 'R eye'
  'R eye' 'R eye' 'L ear' 'L ear' 'L ear' 'R ear' 'R ear' 'R ear'
  'tail base' 'tail base' 'tail base']
 ['coords' 'x' 'y' 'likelihood' 'x' 'y' 'likelihood' 'x' 'y' 'likelihood'
  'x' 'y' 'likelihood' 'x' 'y' 'likelihood' 'x' 'y' 'likelihood']]
AD2205_Day3_Rat_13DLC_resnet50_BIAfinal22Apr11shuffle1_500000.csv
49


In [11]:
# code from Corina
for csv in csv_list: 
  #print(csv)
  #convert items from object to ints
  for row in range(2,csv.shape[0]):
    for col in range(1,csv.shape[1]):
      csv[row,col] = float(csv[row,col])

# Variables for Low Probability Correciton and Freezing


In [12]:
csv = pd.read_csv('/content/drive/MyDrive/BIA_folder/csvs/AD2205_Day2_Rat_1DLC_resnet50_BIAfinal22Apr11shuffle1_500000.csv', dtype='a').to_numpy()

column_names = ['nose_x', 'nose_y', 'nose_likelihood',
                'l_eye_x', 'l_eye_y', 'l_eye_likelihood',
                'r_eye_x', 'r_eye_y', 'r_eye_likelihood',
                'l_ear_x', 'l_ear_y', 'l_ear_likelihood',
                'r_ear_x', 'r_ear_y', 'r_ear_likelihood',
                'tail_base_x', 'tail_base_y', 'tail_base_likelihood']


#convert items from object to ints
for row in range(2,csv.shape[0]):
  for col in range(1,csv.shape[1]):
    csv[row,col] = float(csv[row,col])

df_csv = pd.DataFrame(csv[2:,1:], columns = column_names) 

display(df_csv)

# print(df_csv['nose_likelihood'].mean())


,nose_x,nose_y,nose_likelihood,l_eye_x,l_eye_y,l_eye_likelihood,r_eye_x,r_eye_y,r_eye_likelihood,l_ear_x,l_ear_y,l_ear_likelihood,r_ear_x,r_ear_y,r_ear_likelihood,tail_base_x,tail_base_y,tail_base_likelihood
0,353.596649,164.722839,0.999983,317.558136,140.07373,0.999999,321.23764,174.695328,1.0,280.01413,129.969269,0.999998,296.483398,183.200867,1.0,290.394409,322.10733,0.000536
1,352.076202,166.077255,0.999113,319.342499,142.332916,1.0,321.85498,177.030655,1.0,282.109283,130.720093,0.999999,296.392029,184.132141,1.0,291.293549,321.148895,0.002453
2,352.809418,172.543396,0.999959,318.482574,148.094254,1.0,319.14328,181.987808,1.0,279.570099,133.959076,1.0,292.630859,185.409378,1.0,291.35788,321.412628,0.014242
3,347.511963,175.567322,0.999997,320.527466,153.600281,1.0,317.533936,182.883118,1.0,283.616302,132.630432,1.0,291.041107,186.218887,1.0,290.415009,322.674377,0.003761
4,348.40506,178.302216,0.999714,323.280701,158.149734,1.0,315.942169,188.082352,0.999964,289.668152,133.018845,1.0,289.894958,186.214264,1.0,290.35849,322.923859,0.017051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9341,334.340454,332.934723,0.999999,336.785461,299.851501,0.992998,301.006866,307.173248,1.0,329.472839,251.993958,1.0,275.890564,269.243774,1.0,271.294739,68.172798,0.999992
9342,330.423859,334.945251,0.999997,336.322327,301.217102,0.967916,300.083771,307.564178,1.0,327.933868,251.686157,1.0,275.140533,270.016571,1.0,272.30365,67.468468,0.999713
9343,337.121155,332.438843,1.0,340.081909,294.738464,0.99619,305.39209,306.346741,1.0,332.865204,251.60968,1.0,279.406097,268.59549,1.0,273.63504,65.220215,0.996964
9344,339.325958,334.779358,1.0,341.843658,299.785217,0.98683,308.31192,306.896759,1.0,336.44223,252.422775,1.0,282.642487,268.237183,1.0,276.711426,66.539772,0.997054


# Low Probability Correction

In [14]:
def replace_by_bp_likelihood(input_csv_df, pixel_treshold, column_names):

  returned_csv_df = input_csv_df   

  for c in range(0,len(column_names),3): # for each body part
    
    good_probability_index = -1 # to keep track of most recent xy cord with high likelihood
    frame = 0 # to iterate through frames 

    x = input_csv_df[column_names[c]].to_numpy() # x column for body part 
    y = input_csv_df[column_names[c+1]].to_numpy() # y column 
    likelihood = input_csv_df[column_names[c+2]].to_numpy() # likelihood 
    
    while frame < len(input_csv_df) - 1: # for each frame

      if likelihood[frame] >= likelihood.mean(): # if the likelihood of that frame's prediction is less than the average for that body part
        good_probability_index = frame # save this frame index as a frame with high prediction likelihood 
        frame += 1 # iterate 

      else: # if the frame has less than the average likelihood for that body part 
        bad_probability_index = [] # start a list to hold this index and any consecutive indices with less than average likelihood 

        while likelihood[frame] < likelihood.mean(): 
          bad_probability_index.append(frame) # add the indices of the frames with below bp average likelihood to list 
          frame += 1

        # once outside this loop, there will be a list of the frames with too low of likelihoods, so we want to replace the values 

        if bad_probability_index[0] == 0: # if the first frame has too low of a prediction, 
                    # we will replace these first bad frames with the value of the first good frame in the column 
          for i in range(len(bad_probability_index)):
            returned_csv_df[column_names[c]][bad_probability_index[i]] = returned_csv_df[column_names[c]][bad_probability_index[-1] + 1] # x cord
            returned_csv_df[column_names[c+1]][bad_probability_index[i]] = returned_csv_df[column_names[c+1]][bad_probability_index[-1] + 1] # y cord 
        
        elif bad_probability_index[-1] == (len(input_csv_df) - 1): # if the last frame has too low a prediction, 
                    # we will replace these last bad frames with the value of the last good frame in the column 
          for i in range(len(bad_probability_index)):
            returned_csv_df[column_names[c]][bad_probability_index[i]] = returned_csv_df[column_names[c]][good_probability_index] # x cord
            returned_csv_df[column_names[c+1]][bad_probability_index[i]] = returned_csv_df[column_names[c+1]][good_probability_index] # y cord
                            
        else: # somewhere in the middle, use the linear interpolation between the last frame with a likelihood over the bp average, 
                    # and the next frame with a likelihood over the bp average
          
          # next is to make line between the two points, and increment evenly in between 
          start_coord = [x[good_probability_index], y[good_probability_index]]
          finish_coord = [x[bad_probability_index[-1] + 1 ], y[bad_probability_index[-1] + 1 ]]

          x_difference = finish_coord[0] - start_coord[0] # get the x distance and the direction 
          y_difference = finish_coord[1] - start_coord[1] # get the y distance and the direction  

          increment_x = x_difference / (len(bad_probability_index) + 1) # this is the increment to move each point 
          increment_y = y_difference / (len(bad_probability_index) + 1)

          for i in range(len(bad_probability_index)): 
            returned_csv_df[column_names[c]][bad_probability_index[i]] = start_coord[0] + (i+1)*increment_x # create the new values and replace
            returned_csv_df[column_names[c+1]][bad_probability_index[i]] = start_coord[1] + (i+1)*increment_y  

  return returned_csv_df

In [15]:
# test 
print(df_csv['tail_base_likelihood'][923:930].to_numpy())

print(df_csv['tail_base_x'][923:930].to_numpy())

print(df_csv['tail_base_y'][923:930].to_numpy())

x = replace_by_bp_likelihood(df_csv, 10, column_names)

[0.9865080118179321 0.8125309944152832 0.18080651760101318
 0.015712082386016846 0.06705260276794434 0.6628450155258179
 0.8418127298355103]
[359.19439697265625 359.05670166015625 360.1405944824219 367.1399841308594
 306.40936279296875 308.4866943359375 363.54364013671875]
[210.50338745117188 82.1769790649414 209.82022094726562 204.9342041015625
 236.99143981933594 236.79452514648438 81.3436279296875]


In [16]:
print(x['tail_base_likelihood'][923:930].to_numpy())

print(x['tail_base_x'][923:930].to_numpy())

print(x['tail_base_y'][923:930].to_numpy())

[0.9865080118179321 0.8125309944152832 0.18080651760101318
 0.015712082386016846 0.06705260276794434 0.6628450155258179
 0.8418127298355103]
[359.19439697265625 359.05670166015625 346.41419982910156
 333.7716979980469 321.1291961669922 308.4866943359375 363.54364013671875]
[210.50338745117188 82.1769790649414 120.83136558532715 159.4857521057129
 198.14013862609863 236.79452514648438 81.3436279296875]


#Freezing versus Motion

## Nose Freeze


In [18]:
def nose_freezing(input_csv, pixel_threshold, column_names):

  differences = [] # raw frame to frame xy cord distance
  prev_cord = [0,0]
  freeze_detected_in_frame = [] # 0 and 1s for no and yes 

  df_csv = pd.DataFrame(input_csv, columns = column_names) 

  for frame, data in df_csv.iterrows(): # iterate through the rows of the csv
    nose_cord = [data.nose_x, data.nose_y] # get the x and y coord for the body part 
    if frame == 0: # handle the first frame
      prev_cord = nose_cord
      differences.append(0)
      distance = pixel_threshold * 2 # so that freeze will be 0 
    else: 
      distance = ((nose_cord[0] - prev_cord[0])**2 + (nose_cord[1] - prev_cord[1])**2)**0.5 # get the xy difference in frame to frame position for that body part 
      differences.append(distance)
      prev_cord = nose_cord # set the current coord to the prev coord for the calculation in the next loop

    if distance < pixel_threshold: # if the distance calculated between frames is less than the threshold specified by the user
      freeze_detected_in_frame.append(1) # this is freexing 
    else:
      freeze_detected_in_frame.append(0) # not freezing 
    
  return freeze_detected_in_frame

test = nose_freezing(df_csv, 2, column_names)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

The freezing functions work exactly the same as the nose freeze function but handle the specific body parts

## Right Eye Freezing

In [19]:
def r_eye_freezing(input_csv, pixel_threshold, column_names):

  differences = [] # raw frame to frame xy cord distance
  prev_cord = [0,0]
  freeze_detected_in_frame = [] # 0 and 1s for no and yes 

  df_csv = pd.DataFrame(input_csv, columns = column_names) 

  for frame, data in df_csv.iterrows():
    r_eye_cord = [data.r_eye_x, data.r_eye_y]
    if frame == 0:
      prev_cord = r_eye_cord
      differences.append(0)
      distance = pixel_threshold * 2 # so that freeze will be 0 
    else: 
      distance = ((r_eye_cord[0] - prev_cord[0])**2 + (r_eye_cord[1] - prev_cord[1])**2)**0.5
      differences.append(distance)
      prev_cord = r_eye_cord

    if distance < pixel_threshold:
      freeze_detected_in_frame.append(1)
    else:
      freeze_detected_in_frame.append(0)
    
  return freeze_detected_in_frame

test = r_eye_freezing(df_csv, 2, column_names)

[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

##Left Eye Freezing

In [20]:
def l_eye_freezing(input_csv, pixel_threshold, column_names):

  differences = [] # raw frame to frame xy cord distance
  prev_cord = [0,0]
  freeze_detected_in_frame = [] # 0 and 1s for no and yes 

  df_csv = pd.DataFrame(input_csv, columns = column_names) 

  for frame, data in df_csv.iterrows():
    l_eye_coord = [data.l_eye_x, data.l_eye_y]
    if frame == 0:
      prev_cord = l_eye_coord
      differences.append(0)
      distance = pixel_threshold * 2 # so that freeze will be 0 
    else: 
      distance = ((l_eye_coord[0] - prev_cord[0])**2 + (l_eye_coord[1] - prev_cord[1])**2)**0.5
      differences.append(distance)
      prev_cord = l_eye_coord

    if distance < pixel_threshold:
      freeze_detected_in_frame.append(1)
    else:
      freeze_detected_in_frame.append(0)
    
  return freeze_detected_in_frame

test = l_eye_freezing(df_csv, 2, column_names)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 

## Right Ear Freeze

In [34]:
def r_ear_freezing(input_csv, pixel_threshold, column_names):

  differences = [] # raw frame to frame xy cord distance
  prev_cord = [0,0]
  freeze_detected_in_frame = [] # 0 and 1s for no and yes 

  df_csv = pd.DataFrame(input_csv, columns = column_names)  

  for frame, data in df_csv.iterrows():
    r_ear_coord = [data.r_ear_x, data.r_ear_y]
    if frame == 0:
      prev_cord = r_ear_coord
      differences.append(0)
      distance = pixel_threshold * 2 # so that freeze will be 0 
    else: 
      distance = ((r_ear_coord[0] - prev_cord[0])**2 + (r_ear_coord[1] - prev_cord[1])**2)**0.5
      differences.append(distance)
      prev_cord = r_ear_coord
    if distance < pixel_threshold:
      freeze_detected_in_frame.append(1)
    else:
      freeze_detected_in_frame.append(0)

  return freeze_detected_in_frame

test = r_ear_freezing(df_csv, 2, column_names)

[0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

## Left Ear Freeze




In [31]:
def l_ear_freezing(input_csv, pixel_threshold, column_names):

  differences = [] # raw frame to frame xy cord distance
  prev_cord = [0,0]
  freeze_detected_in_frame = [] # 0 and 1s for no and yes 

  df_csv = pd.DataFrame(input_csv, columns = column_names)  

  for frame, data in df_csv.iterrows():
    l_ear_cord = [data.l_ear_x, data.l_ear_y]
    if frame == 0:
      prev_cord = l_ear_cord
      differences.append(0)
      distance = pixel_threshold * 2 # so that freeze will be 0 
    else: 
      distance = ((l_ear_cord[0] - prev_cord[0])**2 + (l_ear_cord[1] - prev_cord[1])**2)**0.5
      differences.append(distance)
      prev_cord = l_ear_cord
    if distance < pixel_threshold:
      freeze_detected_in_frame.append(1)
    else:
      freeze_detected_in_frame.append(0)

  return freeze_detected_in_frame

test = l_ear_freezing(df_csv, 2, column_names)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

## Tail Base Freeze

In [32]:
def tail_base_freezing(input_csv, pixel_threshold, column_names):

  differences = [] # raw frame to frame xy cord distance
  prev_cord = [0,0]
  freeze_detected_in_frame = [] # 0 and 1s for no and yes 

  df_csv = pd.DataFrame(input_csv, columns = column_names) 

  for frame, data in df_csv.iterrows():
    tail_cord = [data.tail_base_x, data.tail_base_y]
    if frame == 0:
      prev_cord = tail_cord
      differences.append(0)
      distance = pixel_threshold * 2 # so that freeze will be 0 
    else: 
      distance = ((tail_cord[0] - prev_cord[0])**2 + (tail_cord[1] - prev_cord[1])**2)**0.5
      differences.append(distance)
      prev_cord = tail_cord
    if distance < pixel_threshold:
      freeze_detected_in_frame.append(1)
    else:
      freeze_detected_in_frame.append(0)
    
  return freeze_detected_in_frame

test = tail_base_freezing(df_csv, 2, column_names)

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 

## Body Freeze


In [37]:
def body_freeze(input_csv, pixel_threshold, column_names, window_of_frames):

  freeze_window = [] # to hold the true false value if a full body freeze is detected 

  low_probability_correction_input_csv = replace_by_bp_likelihood(input_csv, pixel_threshold, column_names) # clean the data using my linear interpolation function 

  nose = nose_freezing(low_probability_correction_input_csv, pixel_threshold, column_names) # get the freeze arrays for each body part, using user specified pixel threshold 
  r_eye = r_eye_freezing(low_probability_correction_input_csv, pixel_threshold, column_names)
  l_eye = l_eye_freezing(low_probability_correction_input_csv, pixel_threshold, column_names)
  r_ear = r_ear_freezing(low_probability_correction_input_csv, pixel_threshold, column_names)
  l_ear = l_ear_freezing(low_probability_correction_input_csv, pixel_threshold, column_names)
  tail = tail_base_freezing(low_probability_correction_input_csv, pixel_threshold, column_names)

  for i in range(len(input_csv)): # for each frame
    if np.all(nose[i:i+window_of_frames]) and np.all(r_eye[i:i+window_of_frames]) and np.all(l_eye[i:i+window_of_frames]) and np.all(r_ear[i:i+window_of_frames]) and np.all(l_ear[i:i+window_of_frames]) and np.all(tail[i:i+window_of_frames]):
        # freezing in all body parts found from ith frame for the next user specified consecutive window_of_frames frames 
        freeze_window.append(True)
    else:
      freeze_window.append(False) # not freezing 
  
  return freeze_window

final = body_freeze(df_csv, 2, column_names, 15) # test with one video, 2 pixel threshold, and 15 consecutive frames 

print(final)
print(sum(final))

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False